# Gemini as a Customer Service Agent demo 

## Data Preparation

Start by using BigQuery's integration with Google Cloud's ML and AI APIs to prepare your data for submission to the Gemini model.

### Translate

Use the following query to translate your queries from their raw text into a unified language (in this case, English) using the [Cloud Translation API](https://cloud.google.com/translate). This doesn't require us to know the source language, but simply define the target language and continue. Check out the documentation [learn more](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-translate) about using the `ML.TRANSLATE` function in BigQuery, including additional use cases.

In [ ]:
%%bigquery results
CREATE OR REPLACE TABLE `data-quality-demo-next-24.cymbal_sports.reviews_joined_translated`
AS
SELECT
  reviews.* EXCEPT (review_text, text_content,ml_translate_result, ml_translate_status),
  review_text AS review_text_raw,
  REGEXP_REPLACE(TRIM(JSON_VALUE(ml_translate_result, '$.translations[0].translated_text')), r'([a-zA-Z0-9\s]*)&#39;([a-zA-Z0-9\s]*)', "\\1'\\2") AS review_text,
  language_name_en AS review_language,
FROM
  ML.TRANSLATE(MODEL `data-quality-demo-next-24.cymbal_sports.translate`,
    (
    SELECT
      *,
      review_text AS text_content
    FROM
     `data-quality-demo-next-24.cymbal_sports.raw_reviews_joined`),
    STRUCT('translate_text' AS translate_mode, 'en' AS target_language_code)
  ) reviews
LEFT JOIN
  `data-quality-demo-next-24.cymbal_sports.iso_639_codes` iso ON TRIM(JSON_VALUE(reviews.ml_translate_result, '$.translations[0].detected_language_code')) = iso.iso_639_1

In [ ]:
results

### Sentiment analysis

Run the following query to analyze the sentiment of each review using the [Cloud Natural Language API](https://cloud.google.com/natural-language). This allows us to determine the sentiment of each review, as well as the magnitude of this sentiment. We can break this down further too in order to understand which sentences express the sentiment most strongly, and more. Check out the documentation [learn more](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-understand-text) about using the `ML.UNDERSTAND_TEXT` function in BigQuery, including additional use cases.

In [ ]:
%%bigquery results
CREATE OR REPLACE TABLE `data-quality-demo-next-24.cymbal_sports.cleaned_reviews`
AS
SELECT
  * EXCEPT (text_content, ml_understand_text_result, ml_understand_text_status),
  CASE
    WHEN CAST(JSON_VALUE(ml_understand_text_result, '$.document_sentiment.score') AS FLOAT64) > 0 THEN "positive"
    WHEN CAST(JSON_VALUE(ml_understand_text_result, '$.document_sentiment.score') AS FLOAT64) < 0 THEN "negative"
    WHEN CAST(JSON_VALUE(ml_understand_text_result, '$.document_sentiment.score') AS FLOAT64) = 0 THEN "neutral"
    ELSE "unknown"
  END AS sentiment,
  CAST(JSON_VALUE(ml_understand_text_result, '$.document_sentiment.magnitude') AS FLOAT64) AS sentiment_magnitude,
  CAST(JSON_VALUE(ml_understand_text_result, '$.document_sentiment.score') AS FLOAT64) AS sentiment_score,
FROM
  ML.UNDERSTAND_TEXT(MODEL `data-quality-demo-next-24.cymbal_sports.nlp`,
    (
    SELECT
      *,
      review_text AS text_content
    FROM
      `data-quality-demo-next-24.cymbal_sports.reviews_joined_translated`),
    STRUCT('ANALYZE_SENTIMENT' AS nlu_option)
  )

In [ ]:
results

### Text Parsing

Parse the text from your customer service policy (a PDF exported to PNGs) into BigQuery using the [Vision AI API](https://cloud.google.com/vision). This performs OCR (Optical Character Recognition) and returns the text into a single cell that we can pass to Gemini later. This can also be done using [Document AI](https://cloud.google.com/document-ai), which is particularly useful for bringing data from highly-structured documents (like tax forms and driver's licenses) into BigQuery. Check out the documentation [learn more](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-annotate-image) about using the `ML.ANNOTATE_IMAGE` function in BigQuery, including additional use cases.

In [ ]:
%%bigquery results
CREATE OR REPLACE TABLE `data-quality-demo-next-24.cymbal_sports.complete_service_policy`
AS
with hold AS(
  SELECT
    REGEXP_REPLACE(REGEXP_REPLACE(JSON_VALUE(ml_annotate_image_result.full_text_annotation.text), r'\n', ' '), r'•|●', '') AS text_content,
    CAST(REGEXP_EXTRACT(uri, r'^gs\:\/\/customer-service-policy-us/images/Cymbal Sports Customer Service Policy_Page_([0-9]{2})\.jpg$') AS INT64) AS page_number,
    *
  FROM
    ML.ANNOTATE_IMAGE(MODEL `data-quality-demo-next-24.cymbal_sports.vision_ai`,
      TABLE `data-quality-demo-next-24.cymbal_sports.customer_service_policy`,
      STRUCT(['DOCUMENT_TEXT_DETECTION'] AS vision_features)) reviews)

SELECT ARRAY_TO_STRING(ARRAY(SELECT TRIM(text_content, "Internal Only For use by Cymbal Sports employees only") FROM hold WHERE page_number > 2 ORDER BY page_number)," ") AS service_policy_text, 1.0 AS version_number

## Resolving customer issues

Use the following query to call the Python [remote function](https://cloud.google.com/bigquery/docs/remote-functions) to pass the appropriate information to the Gemini model and get back a recommendation for action to resolve the issue, plus recommended communications to the user in both the review language and English. 

In [ ]:
%%bigquery results

WITH
  hold AS (
  SELECT
    PARSE_JSON(`data-quality-demo-next-24.cymbal_sports_lineage.analyze_data` (review_id)) AS response
  FROM
    `data-quality-demo-next-24.cymbal_sports.cleaned_reviews`
  WHERE
    sentiment = "negative"
    AND uri IS NOT NULL )

SELECT
  STRING(response.issue_resolution) AS issue_resolution,
  STRING(response.response_user_language) AS email_user_language,
  STRING(response.response_translated) AS email_translated
FROM
  hold

In [ ]:
results

## Review usage lineage

This demo also highlights how BigQuery and Google Cloud's Generative AI tooling can help practitioners capture and review their team's AI usage over time. [Cloud Pub/Sub](https://cloud.google.com/pubsub) is used to capture metadata from Gemini invocations, including the prompt & model response, version of the model and customer service policy used, and embeddings of key inputs that support multimodal similarity search in BigQuery that allow us to identify patterns over time.

### Clean and review prompt data

Let's start by looking at the information for prompts that were sent to Gemini. Cloud Pub/Sub captures this data and writes it directly to BigQuery. Here we'll use BigQuery's [native support for JSON](https://cloud.google.com/bigquery/docs/json-data) to parse out the data from Pub/Sub and review it. You can uncomment the first 2 lines of the query to save these to a BigQuery table, or orchestrate these queries on an ongoing basis using [Dataform](https://cloud.google.com/dataform). 

In [ ]:
%%bigquery results

-- CREATE OR REPLACE TABLE `data-quality-demo-next-24.cymbal_sports_lineage.cleaned_prompts`
-- AS

with parser AS (
  SELECT
    * EXCEPT(data),
    PARSE_JSON(data, wide_number_mode=>'round') AS data
  FROM
    `data-quality-demo-next-24.cymbal_sports_lineage.prompts`
),

cleaning AS (
  SELECT
    data AS original_message,
    publish_time,
    JSON_VALUE(data.review_id) AS review_id,
    TRIM(JSON_VALUE(data.prompt)) AS prompt,
    JSON_VALUE(data.model_version) AS model_version,
    JSON_VALUE(data.policy_version) AS policy_version,
    subscription_name,
    attributes,
    CONCAT("[",TRIM(JSON_VALUE(data.text_embed),"[]"),"]") AS prompt_embedding,
    CONCAT("[",TRIM(JSON_VALUE(data.review_embed),"[]"),"]") AS review_embedding,
    CONCAT("[",TRIM(JSON_VALUE(data.image_embed),"[]"),"]") AS image_embedding,
  FROM
    parser
),

hold AS (
  SELECT
    * EXCEPT(prompt_embedding, review_embedding, image_embedding),
    ROUND(TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), publish_time, MINUTE)/60,2) AS hours_since_prompt,
    prompt_embedding,
    review_embedding,
    image_embedding
  FROM
    cleaning
  WHERE
    prompt_embedding IS NOT NULL
)

SELECT * FROM hold;

In [ ]:
results

### Clean and review response data

In [ ]:
%%bigquery results

-- CREATE OR REPLACE TABLE `data-quality-demo-next-24.cymbal_sports_lineage.cleaned_responses`
-- AS

WITH parser AS (
  SELECT
    * EXCEPT(data),
    PARSE_JSON(data, wide_number_mode=>'round') AS data
  FROM
    `data-quality-demo-next-24.cymbal_sports_lineage.responses`
  ),

cleaning AS (
  SELECT
    TO_JSON(data) AS original_message,
    publish_time,
    JSON_VALUE(data.review_id) AS review_id,
    JSON_VALUE(data.response) AS response,
    (data.safety_attributes) AS safety_attributes,
    message_id,
    subscription_name,
    attributes,
    CONCAT("[",TRIM(JSON_VALUE(data.embedding),"[]"),"]") AS response_embedding,
  FROM
    parser
),

hold AS(
  SELECT
    * EXCEPT(response_embedding),
    ROUND(TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), publish_time, MINUTE)/60,2) AS hours_since_prompt,
    response_embedding
  FROM
    cleaning
  WHERE
    response_embedding IS NOT NULL
)

SELECT * FROM hold


In [ ]:
results